# Mixed-Diffusion Trajectories

In [ ]:
from benchmarks import *
from utilities import *
from competitors import *
from experiments import *
from experiments.save_results import *
from visualization import *
from mdt import *
import numpy as np

In [ ]:
## Load a dataset
dataset = {"name": "isolet_lindenbaum", "noise_factor": None}
Xp, Xv, y = load_preprocessed_dataset(dataset["name"], return_views=True)
Xk = [get_kernel_matrix(x, normalize=False) for x in Xv]
n_clust = len(np.unique(y))

In [ ]:
## Compute singular entropies of a dataset
## Elbows are obtained for t up to 50 generally, depending on the dataset. Some elbows are tuned by hand when KNeed fail to find a good one.
entropies_df = compute_entropy(dataset["name"], t_max = 50)
dataset_name = dataset["name"]
noise_factor = dataset.get("noise_factor", None)
plot_singular_entropy(
    dataset_name,
    noise_factor=noise_factor,
    max_time=25,
    elbow=True,
    path="tables/singular_entropy",
    save=True,
)

In [ ]:
## Create an operator (here: alternating diffusion)
# print(method_list()) # uncomment to see all available methods
operator = alternating_diffusion(Xp)
score = evaluate_operator(
        operator,
        Xp, # list of matrices on which to compute internal criterion
        y,
        metric="ami",
        n_clusters=n_clust,
        embedded=False,
        method="eigen",
        dim_embedd=n_clust,
    )
print(score)

In [ ]:
## Plot embeddings

os.makedirs("figures/embeddings/" + dataset_name, exist_ok=True)
for method in method_list():
    print(method["name"])
    if method.get("single_view", False) or method["name"] == "Direct MDT":
        continue
    if method.get("task", "") == "clustering":
        continue
    A = get_operator_from_method(method, dataset_name, Xp, Xv)
    A_emb = get_embedding(A, dim_embedd = 2, method = method["decomp_method"])
    Embedding_AD = A_emb[:len(y), :2]
    plot_embedding(Embedding_AD, y, name ="/"+ dataset_name +"/" + method["name"] + "_" + method["decomp_method"], type = "manifold", show = False)